# Tissue Classification using Neural Networks
In this lab we will explore the use of texture in images and traditional machine learning approaches such as clustering. The dataset we will be using is available here: http://dx.doi.org/10.5281/zenodo.53169. 

![alt text](https://www.researchgate.net/profile/Jakob_Kather/publication/303998214/figure/fig7/AS:391073710002224@1470250646407/Representative-images-from-our-dataset-Here-the-first-10-images-of-every-tissue-class.png)

The above figure shows the 8 different classes of tissue we will be trying to identify. 

In [0]:
# Imports
from __future__ import print_function
import os
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

## Step 1
* Load the data (done for you)
 * The "data" variable stores 5000 images of shape 150x150. This means data has shape (5000, 150, 150). These images are loaded here as grayscale.
 * The "labels" variable stores 5000 labels (0-7). This means "labels" has shape (5000,)
* Split data into training and testing subsets (left up to you)
 * Check out the sklearn function train_test_split from sklearn.model_selection

In [3]:
! git clone https://github.com/BeaverWorksMedlytics/Week3_public.git

# Build the path to the data folder. No need to change directories
# There are a total of 6 files you will have to load
data_dir = os.path.join( os.getcwd(), 'Week3_public', 'data', 'crc')

fatal: destination path 'Week3_public' already exists and is not an empty directory.


In [73]:
# Load data and split into training, testing sets
y = np.load(os.path.join(data_dir, 'rgb01.npz'))
labels = y['labels'][0:500]
data = y['rgb_data']
data = data[0:500,:,:,0:3]
label_str = y['label_str']
label_str = label_str.tolist() # this is to convert label_str back to a dictionary
y = []

print(data.shape)
for ii in range(2,6):
    filename = os.path.join(data_dir, 'rgb0' + str(ii) + '.npz')
    print('loading ', filename)
    y = np.load(filename)
    labels = np.append(labels, y['labels'][0:500], axis=0)
    data = np.append(data, y['rgb_data'][0:500,:,:,0:3], axis=0)
    print(data.shape)
    y = []


print( data.shape )
print( labels.shape )

(500, 150, 150, 3)
loading  /content/Week3_public/data/crc/rgb02.npz
(1000, 150, 150, 3)
loading  /content/Week3_public/data/crc/rgb03.npz
(1500, 150, 150, 3)
loading  /content/Week3_public/data/crc/rgb04.npz
(2000, 150, 150, 3)
loading  /content/Week3_public/data/crc/rgb05.npz
(2500, 150, 150, 3)
(2500, 150, 150, 3)
(2500,)


In [74]:
num_images, nrows, ncols, ncolors = data.shape

# split into training and testing sets
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2, random_state=0)
# convert the labels from 1-D arrays to categorical type 
#labels = to_categorical(labels, num_classes=8)
labels_test= to_categorical(labels_test)
labels_train = to_categorical(labels_train)

print(labels_train.shape)
print(labels_test.shape)

(2000, 8)
(500, 8)


## Normalize data
All images should be normalized to the range 0-1 by dividing by 255.

#### Note
* Using the La\*b colorspace : If you convert your images to the La\*b colorspace, the scaling factor will change. Each channel in this colorspace will have a different range and normalization of each space will involve scaling each channel separately. Additionally, the a\* channel can have a negative range. This also needs to be taken into account. 
* Using the HSV/HSI colorspace : Similar considerations apply if you are using the HSV/HSI colorspace. The only difference is that the HSV/HSI colorspace will have all positive values.

In [75]:
# Assuming we are using the RGB colorspace
# Normalize all images so that they are 0-1
data_train = data_train.astype(float)/255.   #put a period so it knows its an integer
print (data.shape)


(2500, 150, 150, 3)


## Step 2
At this point, the data has been split into training and testing sets and normalized. We will now design a fully connected neural network for texture classification. 

<img src="https://github.com/hkaur2001/Week3_public/blob/master/Notebooks/images/CNN-example-block-diagram.jpg?raw=1" width="100%"></img>

( Image from http://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/ )

When designing a fully connected network for classification, we have several decisions to make.

**Network Architecuture**
* How many layers will our network have ?
* How many convolutional filters per layer ?
    * What is an appropriate filter size ? 
* What is an appropriate batch size, learning rate and number of training epochs ?

**Data input**
* Do we use the raw data ?
    * RGB or just gray channel ?
* Does the use of different colorspaces lead to better results for a given network architecture ?
* Can we use any of the texture features from the previous lab as inputs to this model ?
* How does data augmentation affect the results ? 

Other considerations, we will not be exploring :
* What is the trade-off between input data sizes and batch size ?
* Is the GPU always the appropriate platform for training ?
* How does hardware influence inputs and batch sizes for a given desired accuracy ?

In [0]:
# Define the data shapes based on your decision to use rgb or grayscale or other colorpsaces or texture features or 
# some combination of these inputs
num_classes = 8 
input_shape = nrows*ncols*ncolors

## Step 3
Design your network here using Keras

In [77]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, Flatten

# Create your network
model = []
model = Sequential()

# Add input layer
model.add(Conv2D(12, kernel_size=(5,5), input_shape = data_train[0].shape)) #data_train.shape[0:3]


model.add(GlobalMaxPooling2D())
model.add(Flatten())
model.add(Dense(8, activation= 'sigmoid'))


#keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

# Add fully connected layers 
# See keras.io for Conv2D, MaxPool2D, Dropout documentation

# Add final output layer - This should have as many neurons as the number
# of classes we are trying to identify

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 146, 146, 12)      912       
_________________________________________________________________
global_max_pooling2d_11 (Glo (None, 12)                0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 12)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 104       
Total params: 1,016
Trainable params: 1,016
Non-trainable params: 0
_________________________________________________________________


In [78]:
print(labels_train.shape)

(2000, 8)


## Step 4
Compile the model you designed. Compiltation of the Keras model results in the initialization of model weights and sets other model properties.

In [0]:
import tensorflow as tf

model.compile(loss = 'categorical_crossentropy', optimizer=tf.train.AdamOptimizer(learning_rate=0.01),metrics=['accuracy'])

In [85]:
print(data_train.shape)
print(labels_train.shape)

(2000, 150, 150, 3)
(2000, 8)


## Step 5
Train model

In [86]:
y = model.fit(data_train, labels_train, epochs=5)

Epoch 1/5
2000/2000 [==============================] - 76s 38ms/step - loss: 1.6930 - acc: 0.3350
Epoch 2/5
2000/2000 [==============================] - 77s 38ms/step - loss: 1.2604 - acc: 0.5175
Epoch 3/5
2000/2000 [==============================] - 77s 38ms/step - loss: 1.1187 - acc: 0.5690
Epoch 4/5
2000/2000 [==============================] - 77s 38ms/step - loss: 1.0256 - acc: 0.6210
Epoch 5/5
2000/2000 [==============================] - 77s 38ms/step - loss: 0.9600 - acc: 0.6430


## Step 6
See how your model performs by uisng it for inference.
* What is the accuracy of classification ?
* Change your model, re-compile and test. Can you improve the accuracy of the model ?


In [0]:
# predict labels - use the test set for prediction
pred_labels = model.predict(data_test)

In [94]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# We need to convert the categorical array test_labels into a vector
# in order to use it in the calculation of the confusion matrix
mat = confusion_matrix(np.argmax(labels_test, axis=1), pred_labels)
acc = accuracy_score(np.argmax(labels_test, axis=1), pred_labels)
print(acc)
print(mat)

ValueError: ignored

In [0]:
plt.figure(figsize=(8,6))
plt.imshow(mat, cmap='hot', interpolation='nearest')
plt.grid(False)
plt.colorbar()
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

## Assignment
* In Step 3 design your own network
* Does the model perform better if you use all three RGB channels ?
* How does the performance change when using the La*b colorspace ?
